# Détection automatique de maladies pulmonaires à partir de radiographies

### Pré-traitement des données

In [ ]:
import os
import random
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision.models import ResNet18_Weights

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import tkinter as tk
import customtkinter as ctk
from tkinter import filedialog
from PIL import ImageTk
import torch.nn.functional as F

In [6]:
def load_model(model_path='best_model.pth'):
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = torch.nn.Linear(model.fc.in_features, 3)  # 3 classes
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [7]:
classes = ['Normal', 'COVID', 'Pneumonia']

In [8]:
def predict_image(img_path, model):
    transform = ResNet18_Weights.DEFAULT.transforms()
    image = Image.open(img_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = F.softmax(outputs, dim=1)
        prob, predicted = torch.max(probabilities, 1)
        class_name = classes[predicted.item()]
        confidence = prob.item() * 100  # en %
        return class_name, confidence

In [9]:
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

In [24]:
class LungApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Projet")
        self.geometry("700x800")
        self.resizable(False, False)

        self.model = load_model()
        self.default_progress_color = "#6fa3f7"

        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(1, weight=1)

        self.create_widgets()
        
    def create_widgets(self):
        """Crée tous les widgets de l'interface"""
        # Frame d'en-tête
        header_frame = ctk.CTkFrame(self,height=40, corner_radius=10)
        header_frame.grid(row=0, column=0, sticky="nsew", padx=20, pady=10)
        
        # Titre principal
        self.label_title = ctk.CTkLabel(
            header_frame, 
            text="Détection de maladies pulmonaires", 
            font=("Roboto", 25, "bold"),
            text_color="#6fa3f7"
        )
        self.label_title.pack(pady=(10, 0))
        
        
        # Frame principale
        main_frame = ctk.CTkFrame(self, corner_radius=15)
        main_frame.grid(row=1, column=0, sticky="nsew", padx=20, pady=(0, 20))
        
        # Bouton pour charger l'image
        self.button = ctk.CTkButton(
            main_frame,
            text="Choisir une radiographie",
            command=self.load_image,  
            font=("Roboto", 16, "bold"),
            text_color="#6fa3f7",
            height=50,
            corner_radius=10,
            fg_color="#0d2957",
            hover_color="#0d2957"
        )
        self.button.pack(pady=30, padx=50, fill="x")
        
        # Zone d'affichage de l'image
        self.image_frame = ctk.CTkFrame(main_frame, width=400, height=400, corner_radius=15)
        self.image_frame.pack(pady=(0, 20))
        
        self.image_label = ctk.CTkLabel(
            self.image_frame, 
            text="Aucune image sélectionnée", 
            font=("Roboto", 14),
            text_color="#6fa3f7"
        )
        self.image_label.pack(expand=True, fill="both", padx=10, pady=10)
        
        # Frame des résultats
        self.result_frame = ctk.CTkFrame(main_frame, corner_radius=15, fg_color="transparent")
        self.result_frame.pack(fill="x", padx=50, pady=(0, 30))
        
        
        self.result_text = ctk.CTkLabel(
            self.result_frame,
            text="",
            font=("Roboto", 25, "bold"),
            text_color="#681910"
        )
        
        self.confidence_text = ctk.CTkLabel(
            self.result_frame,
            text="",
            font=("Roboto", 16),
            text_color="#6fa3f7"
        )
        
        # Barre de progression
        self.progress_bar = ctk.CTkProgressBar(
            self.result_frame,
            orientation="horizontal",
            height=15,
            corner_radius=10,
            progress_color=self.default_progress_color
        )
        
        # Footer
        footer_frame = ctk.CTkFrame(self, corner_radius=10, height=40)
        footer_frame.grid(row=2, column=0, sticky="nsew", padx=20, pady=(0, 10))
        
        footer_label = ctk.CTkLabel(
            footer_frame,
            text="© 2025 Aix-Marseille Université - Projet de fin de Licence Informatique",
            font=("Roboto", 10),
            text_color="#7f8c8d"
        )
        footer_label.pack(pady=10)
    
    def load_image(self):
        """Charge et affiche l'image sélectionnée avec les résultats"""
        path = filedialog.askopenfilename(filetypes=[("Images", "*.png *.jpg *.jpeg")])
        if path:
            try:
                self.progress_bar.configure(progress_color=self.default_progress_color)
                self.progress_bar.set(0)
                
                # Afficher un indicateur de chargement
                self.image_label.configure(text="Analyse en cours...", text_color="#3498db")
                self.update()
                
                # Charger et afficher l'image
                img = Image.open(path).resize((350, 350))
                img_tk = ImageTk.PhotoImage(img)
                self.image_label.configure(image=img_tk, text="")
                self.image_label.image = img_tk  # Garder une référence
                
                # Faire la prédiction
                result, confidence = predict_image(path, self.model)
                
                # Afficher les résultats
                self.show_results(result, confidence)
                
            except Exception as e:
                self.image_label.configure(text=f"Erreur: {str(e)}", text_color="#e74c3c")
    
    def show_results(self, result, confidence):
        """Affiche les résultats avec des animations"""
        # Afficher les éléments de résultats
        
        self.progress_bar.pack(fill="x", pady=(0, 20))
        self.result_text.pack(pady=(5, 0))
        self.confidence_text.pack(pady=(0, 10))
        

        # Animer la barre de progression
        self.progress_bar.set(0)
        for i in range(0, 101, 5):
            self.progress_bar.set(i/100)
            self.update()
            self.after(30)

        # Mettre à jour les textes
        self.result_text.configure(text=f"Diagnostic: {result}")
        self.confidence_text.configure(text=f"Confiance: {confidence:.1f}%")
        
        
        
        # Changer la couleur en fonction du résultat
        if result == "Normal":
            self.result_text.configure(text_color="#00C050")
            self.progress_bar.configure(progress_color="#00C050")
        else:
            self.result_text.configure(text_color="#A71100")
            self.progress_bar.configure(progress_color="#A71100")


# === Lancer l'application ===
if __name__ == "__main__":
    app = LungApp()
    app.mainloop()